# Ostrofsky's rootN times rootN PIR scheme based on Learning with Errors and Regev Encryption :

Using parameters from the paper: **One server for the price of Two: Simple and Fast Single-Server Private Information Retreival**

Here the number of LWE samples(m) = m

In [13]:
import numpy as np


# parameters
n = 2**10
m = 10 # sqrt(N), number of samples

error_values = {
    "loc": 0,
    "sigma": 6.4,
}
q = 2**23
p = 991 # plaintext modulus
scaling_factor = np.floor(q/p)


print(f"scaling_factor = floor(q/p) = {np.floor(q/p)}")

x = np.zeros(m)
x[1]=1
print(x * scaling_factor)
print(f"x = {x}")

db = np.random.randint(0, 2, (m,m))
print(f"db = {db}")

scaling_factor = floor(q/p) = 8464.0
[   0. 8464.    0.    0.    0.    0.    0.    0.    0.    0.]
x = [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
db = [[372 565 193 621 898 962 620 240 226 213]
 [599 404 548 752  47 944 397 351 602 844]
 [162 762 231 816 683 858 482 173 657 331]
 [925 583 858 482 129 553 663 361 873 850]
 [293 714 308 612 625 286 687 629 429 274]
 [476 470 302 276  65 431 777 651 641 916]
 [664 234 317 967 304 662 630 554 820 114]
 [ 40 140 107 756 417 151 710 987 167 558]
 [593 648 822 188  28 880 136  44 207 225]
 [864 693 710  85 434 694 901 591 540 623]]


In [14]:
#using matrices (when you have m samples)


def sampleLWE(m,n,q,error_values, s):
    A= np.random.randint(0, q, (m,n))
    
    e= np.random.normal(error_values["loc"], error_values["sigma"], (m) )

    print(f"A = {A}")
    print(f"s = {s}")
    # print(f"e = {e}")


    # Computing the dot product and adding the error

    # Outputting the vectors a, s and the value b

    b = (np.dot(A,s)  + e ) % q
    print(f"b = As + e mod q = {b}")
    b_prime = (np.dot(A,s) % q + e)


    return A, b_prime




In [15]:
# print(f"A = ", A)
# print(type(A), A.shape)
# print(f"s = ", s)
# print(type(s), s.shape)
# print(f"e = ", e)
# print(type(e), e.shape)

In [16]:
secret= np.random.randint(0, q, (n))

A, b = sampleLWE(m,n,q,error_values, secret)

A = [[2348434 1482656 3794582 ... 4571387 5195981 5286873]
 [7522687 5087607 5811939 ... 7793064 1657616 7426621]
 [8043383 6331916 1810174 ... 6088790 3142966 2148332]
 ...
 [ 148011  656658 1753195 ... 7570667 4576037 1077199]
 [7883553 8366683 5836804 ... 4625040 8300232 3513764]
 [7031726 5962303 7336280 ... 1808084 3755676 4733082]]
s = [7105606 7613687 3048286 ... 5192019 7457416 3020859]
b = As + e mod q = [6736716. 1181856. 8050416. 7684352.  939506. 4957714.  331730. 7905374.
   76450. 2539444.]


In [17]:
# print(f"as mod q = {np.dot(A, s)% q}")

# print(f"(b = As mod q + e ) mod q = ", b)

# print(f" b_prime = As + e mod q    = {b_prime}")

In [18]:
# Encryption

def regevEncrypt(b, x, p, q):
    scaling_factor = np.floor(q/p)
    c =( b + scaling_factor*x ) 
    c_mod_q = c % q

    

    # print(f"c= As + e + (floor(q/p) * x) \n \\ \
    # = As + e + {scaling_factor}*{x} = {c} = {c_mod_q}")

    return c_mod_q

In [19]:


c = regevEncrypt(b, x, p, q)
print(c)

[6736715.51365367 1190319.93297226 8050416.21808572 7684353.59445417
  939506.67030583 4957714.67491739  331729.46812539 7905375.38049943
   76449.32933107 2539441.6833221 ]


Send Client --- (mu_enc = A, c) ----> Server

In [20]:
# server computations:

# D dot A

# do you reduce the entire thing modulo q or at the end only

DA = np.dot(db,A) % q
Dc = np.dot(db,c) % q

print(DA)
print(Dc)

[[7482587 6081125 6980638 ... 6602940 5178384 1516908]
 [6561588 7893972 4659972 ... 3553601 2441404 6499275]
 [5218530 1948908 6732008 ... 3114088 5897015 6749244]
 ...
 [7492470 3412268 4678648 ... 8146101 5301189  173525]
 [3301869 3708104 1280178 ...  135899  445297 3087011]
 [  37135 7372441 2032433 ...  137763 3803681  649243]]
[2981989.1942749  6576753.14847565 2242547.5001297  4083035.45292664
 5335877.0361557  8197532.8494339  6391142.58605957 6825893.54883194
 2655686.2083931  2551306.22599411]


In [21]:
#Decryption

# print(np.dot(a, s), np.dot(a,s) % q)

def rounding(x, round_off_to):
    """
    Rounds off a number x to the nearest multiple of round_off_to.

    :param x: The number to be rounded off.
    :param round_off_to: The nearest multiple to which x needs to be rounded off.
    :return: The rounded off value.
    """
    return np.round(x / round_off_to) * round_off_to

# def regevDecrpt(A, s, q, p, c):
#     scaling_factor = np.floor(q/p)
#     # a_s_mod_q = np.dot(A,s)%q 
#     # print(f"a_s_modq = as mod q = ", a_s_mod_q)

#     d_hat = (c - np.dot(A, s)) % q 

#     print(f"d_hat = c - as mod q= {c} - {np.dot(A,s)} mod q = ", d_hat)
#     rounded_d = rounding(d_hat, scaling_factor)



#     print(f"rounded d = {rounded_d}")
    
#     d = rounded_d / scaling_factor
#     print(f"d = rounded_d/ floor(q/p)= {rounded_d}/ {scaling_factor} = {d}" )
#     return d






In [22]:
print(DA)
print(Dc)

[[7482587 6081125 6980638 ... 6602940 5178384 1516908]
 [6561588 7893972 4659972 ... 3553601 2441404 6499275]
 [5218530 1948908 6732008 ... 3114088 5897015 6749244]
 ...
 [7492470 3412268 4678648 ... 8146101 5301189  173525]
 [3301869 3708104 1280178 ...  135899  445297 3087011]
 [  37135 7372441 2032433 ...  137763 3803681  649243]]
[2981989.1942749  6576753.14847565 2242547.5001297  4083035.45292664
 5335877.0361557  8197532.8494339  6391142.58605957 6825893.54883194
 2655686.2083931  2551306.22599411]


In [23]:
d_hat = ( Dc[0] - np.dot(DA[0], secret) ) % q
print(f"d_hat = {d_hat}")

d_hat_prime = ( Dc[0] - ( np.dot(DA[0], secret) % q  ) ) % q
print(f"d_hat_prime = {d_hat}")

rounded_d = rounding(d_hat_prime, scaling_factor)



print(f"rounded d = {rounded_d}")

d = rounded_d / scaling_factor
print(f"d = rounded_d/ floor(q/p)= {rounded_d}/ {scaling_factor} = {d}" )

print(db)

d_hat = 4773516.0
d_hat_prime = 4773516.0
rounded d = 4773696.0
d = rounded_d/ floor(q/p)= 4773696.0/ 8464.0 = 564.0
[[372 565 193 621 898 962 620 240 226 213]
 [599 404 548 752  47 944 397 351 602 844]
 [162 762 231 816 683 858 482 173 657 331]
 [925 583 858 482 129 553 663 361 873 850]
 [293 714 308 612 625 286 687 629 429 274]
 [476 470 302 276  65 431 777 651 641 916]
 [664 234 317 967 304 662 630 554 820 114]
 [ 40 140 107 756 417 151 710 987 167 558]
 [593 648 822 188  28 880 136  44 207 225]
 [864 693 710  85 434 694 901 591 540 623]]


In [24]:
# print(f"x = {x}")
# d = regevDecrpt(A, secret, q, p, c = Dc[0])